# Chapter 08 code snippets
This notebook contains all code snippets from chapter 8.

In [ ]:
print('Hello world')

In [ ]:
# Error message output
invalid_result = 0/0

## Basic coding with the AzureML SDK

In [ ]:
raise NotImplementedError("Please comment out this line and update the values below")
subscription_id = '<Subscription Id>'
resource_group = 'packt-azureml-rg'
workspace_name = 'packt-learning-mlw'

In [ ]:
from azureml.core import Workspace

ws = Workspace(subscription_id, resource_group, workspace_name)

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="<AAD tenant id>")

In [ ]:
from azureml.core import Workspace

ws = Workspace.get(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group)

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(f"Connected to workspace {ws.name}")

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(
               name='packt-azureml-sdk-mlw',
               subscription_id=subscription_id,
               resource_group='packt-azureml-rg',
               create_resource_group=True,
               location='westeurope')

### Deploying compute targets

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(f"Connected to workspace {ws.name}")

In [ ]:
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print(f"Compute {compute.name} is a {type(compute)}")

In [ ]:
from azureml.core import ComputeTarget
from azureml.exceptions import ComputeTargetException

compute_name = 'gpu-cluster'
compute = None
try:
    compute = ComputeTarget(workspace=ws, name=compute_name)
    print(f"Found {compute_name} which is {type(compute)}")
except ComputeTargetException as e:
    print(f"Failed to get compute {compute_name}. Error: {e}")


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_name = 'cpu-sm-cluster'
cluster = None

if compute_name in ws.compute_targets:
    print('Getting reference to compute cluster')
    cluster = ws.compute_targets[compute_name]
else:
    print('Creating compute cluster')
    config = AmlCompute.provisioning_configuration(
                           vm_size='Standard_D1', 
                           max_nodes=5)
    cluster = ComputeTarget.create(ws, compute_name, config)
    cluster.wait_for_completion(show_output=True)

print(f"Got reference to cluster {cluster.name}") 

In [ ]:
from azureml.core.compute import AmlCompute

for ct_name, ct in ws.compute_targets.items():
    if (isinstance(ct, AmlCompute)):
        print(f"Scalling down cluster {ct_name}")
        try:
            ct.update(min_nodes=0)
        except Exception as e:
            print(f"Failed to scale down cluster {ct.name} with error {e})")

### Defining datastores

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(f"Connected to workspace {ws.name}")

In [ ]:
storage_name = 'mydatastg'
storage_key = '<storagekey>'
storage_container = 'existing-container'

In [ ]:
from azureml.core import Datastore

dstore = Datastore.register_azure_blob_container(
    workspace=ws,
    datastore_name="my_data_store",
    container_name=storage_container,
    account_name=storage_name,
    account_key=storage_key,
    create_if_not_exists=False,
)

# If the container doesn't exist you will get an error
# azure.common.AzureMissingResourceHttpError: The specified container does not exist. ErrorCode: ContainerNotFound

In [ ]:
from azureml.core import Datastore

dstore = Datastore.get(ws,"my_data_store")

In [ ]:
dstore = ws.get_default_datastore()

In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd

features, target = load_diabetes(return_X_y=True)
diabetes_df = pd.DataFrame(features)
diabetes_df['target']= target

diabetes_df.to_csv('rawdata.csv',
                    index=False)

dstore = ws.get_default_datastore()

dstore.upload_files(
    files=['rawdata.csv'],
    target_path="/samples/diabetes/v1", 
    overwrite=True,
    show_progress=True)

### Working with datasets

In [ ]:
from azureml.core import Dataset

dstore = ws.get_default_datastore()
file_paths = [
    (dstore, "/samples/diabetes/v1")
]

file_ds = Dataset.File.from_files(
    path = file_paths,
    validate=True
)

print("Files in FileDataset:")
print(file_ds.to_path())

In [ ]:
tabular_dataset = Dataset.Tabular.from_delimited_files(
    path=file_paths,
    validate=False)

df = tabular_dataset.to_pandas_dataframe()
print(len(df))

In [ ]:
tabular_dataset.register(
    workspace=ws,
    name="diabetes",
    description="The sklearn diabetes dataset")

In [ ]:
Dataset.Tabular.register_pandas_dataframe(
    dataframe=df,
    target=(dstore,"/samples/diabetes"),
    name="diabetes",
    description="The sklearn diabetes dataset")


In [ ]:
from azureml.core import Dataset

diabetes_dataset = Dataset.get_by_name(
    workspace=ws,
    name='diabetes')

In [ ]:
partial_dataset = diabetes_dataset \
        .skip(10) \
        .take(2) \
        .keep_columns(['0','target'])

In [ ]:
df = partial_dataset.to_pandas_dataframe()
df.head()

## Working with the Azure CLI

In [ ]:
! az extension add -n azure-cli-ml

In [ ]:
! az login

In [ ]:
! az account set --subscription "<subscription id>"

In [ ]:
! az ml computetarget list -g packt-azureml-rg -w packt-learning-mlw -o table

In [ ]:
! az ml computetarget update amlcompute --name gpu-cluster --min-nodes 0 -g packt-azureml-rg -w packt-learning-mlw

In [ ]:
! az ml datastore show-default -g packt-azureml-rg -w packt-learning-mlw

In [ ]:
! az ml dataset list -g packt-azureml-rg -w packt-learning-mlw -o table